# Expressive Gestures

Laban/Bartenieff Movement Analysis (LMA) characterizes gestures as individual movements that convey meaning. In LMA, expressive gestures are defined as movements that convey emotions, intentions, or personal artistic expression, often characterized by their dynamic qualities, such as flow, weight, time, and space, which reflect the performer's inner state or interpretation of a theme. In this notebook, the same behavioral gestures will be analyzed to extract expressive qualities within these movements.

### Required Modules

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

### Load Data and Define Paths

In [ ]:
df_pose_abs = pd.read_csv('./data/interim/landmarks/contemporary_dance_solo_20230219225812_10.csv')
df_pose_rel = pd.read_csv('./data/interim/landmarks/contemporary_dance_solo_20230219225812_10.csv')
path_gestures_stand = sorted(os.listdir('.data/interim/dance_video_name/gestures/stand/'))
path_gestures_sitdown = sorted(os.listdir('.data/interim/dance_video_name/gestures/sitdown/'))
path_gestures_bentknee = sorted(os.listdir('.data/interim/dance_video_name/gestures/bentknee/'))

## Timing Variations

The aim of this section is to analyze the duration, rhythm, and tempo of the behavioral gestures to identify any variations that might indicate emotional expression. For example, a slower or faster pace in standing or sitting might reflect the performer's mood or intention.

In [ ]:
# Calculate the duration of each gesture type
duration_stand = len(path_gestures_stand)
duration_sitdown = len(path_gestures_sitdown)
duration_bentknee = len(path_gestures_bentknee)

# Calculate the tempo of each gesture type
frame_rate = 24
tempo_stand = duration_stand / frame_rate
tempo_sitdown = duration_sitdown / frame_rate
tempo_bentknee = duration_bentknee / frame_rate

# Calculate the rhythm (time between gestures) assuming uniform spacing between gestures
rhythm_stand = 0 if len(path_gestures_stand) <= 1 else (path_gestures_stand[-1] - path_gestures_stand[0]) / (len(path_gestures_stand) - 1)
rhythm_sitdown = 0 if len(path_gestures_sitdown) <= 1 else (path_gestures_sitdown[-1] - path_gestures_sitdown[0]) / (len(path_gestures_sitdown) - 1)
rhythm_bentknee = 0 if len(path_gestures_bentknee) <= 1 else (path_gestures_bentknee[-1] - path_gestures_bentknee[0]) / (len(path_gestures_bentknee) - 1)

print("Duration (Stand, Sitdown, Bent Knee):", duration_stand, duration_sitdown, duration_bentknee)
print("Tempo (Stand, Sitdown, Bent Knee):", tempo_stand, tempo_sitdown, tempo_bentknee)
print("Rhythm (Stand, Sitdown, Bent Knee):", rhythm_stand, rhythm_sitdown, rhythm_bentknee)

## Flow and Dynamics

The aim of this section is to examine the smoothness, continuity, and intensity of movements during the execution of behavioral gestures, as they may reveal the performer's emotional state. The following code calculates flow values for each frame in the behavioral gestures by measuring the Euclidean distance between the pose keypoints in consecutive frames, and then calculates the mean, standard deviation, minimum, and maximum flow values for each gesture to provide insights into the smoothness, continuity, and intensity of the movements.

In [ ]:
# Define a function to calculate the Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

# Create an empty DataFrame to store the flow and dynamics information
flow_dynamics_df = pd.DataFrame(columns=['Gesture', 'Frame', 'Flow'])

# Loop through each gesture directory and calculate flow values for each frame
gesture_directories = ['stand', 'sitdown', 'bentknee']
for gesture in gesture_directories:
    gesture_path = f'.data/interim/dance_video_name/gestures/{gesture}/'
    image_files = sorted(os.listdir(gesture_path))
    
    prev_coordinates = None
    for i, image_file in enumerate(image_files[:-1]):
        current_frame = int(image_file.split('.')[0])
        next_frame = int(image_files[i + 1].split('.')[0])
        
        current_coordinates = df_pose_abs[df_pose_abs['frame'] == current_frame].iloc[:, 1:].values
        next_coordinates = df_pose_abs[df_pose_abs['frame'] == next_frame].iloc[:, 1:].values
        
        if prev_coordinates is not None:
            flow = euclidean_distance(prev_coordinates, current_coordinates)
            flow_dynamics_df = flow_dynamics_df.append({
                'Gesture': gesture,
                'Frame': current_frame,
                'Flow': flow
            }, ignore_index=True)
        
        prev_coordinates = next_coordinates

# Calculate statistics for each gesture
flow_dynamics_summary = flow_dynamics_df.groupby('Gesture').agg({
    'Flow': ['mean', 'std', 'min', 'max']
}).reset_index()

print(flow_dynamics_summary)

## Body Posture and Alignment

The aim of this section is to assess the posture and alignment of the body during the performance of behavioral gestures. Differences in body posture can indicate the performer's emotions, intentions, or personal style. Here are the potential insights per metric:

* **Mean per relevant joint**: This data can be used to compare other gestures or to the ideal posture to see if there are any significant deviations.
* **Standard deviation per relevant joint**: High variability may indicate that the gesture is more difficult to execute or that the performer is expressing more emotion.
* **Calculated Range of Motion**: This data can be used to compare with other gestures or to the ideal posture to see if there are any significant differences.
* **Average Range of Motion**: Higher average range of motion may indicate that the performer is expressing more emotion or that the gesture is more dynamic.

In [ ]:
# Define the joint columns to use for posture and alignment analysis
standing_joint_columns = [col for col in df_pose_abs.columns if col not in ['frame', 'landmarks', 'torso_x', 'torso_y', 'torso_z']]
standing_joint_mean = df_pose_abs[joint_columns].mean()
standing_joint_std = df_pose_abs[joint_columns].std()
standing_joint_max = df_pose_abs[joint_columns].max()
standing_joint_min = df_pose_abs[joint_columns].min()

# Calculate the range of motion (RoM)
standing_joint_range = standing_joint_max - standing_joint_min
standing_avg_range = np.mean(joint_range)

# Calculate the average posture and alignment values across all frames
standing_posture_mean = np.mean(joint_mean)
standing_alignment_mean = np.mean(joint_mean[['left_hip', 'right_hip']] - joint_mean[['left_shoulder', 'right_shoulder']])

print(f"Mean per relevant joint: {standing_joint_mean})
print(f"STD per relevant joint: {standing_joint_mean})
print(f"Calculated range of motion: {standing_joint_range})
print(f"Average Range of Motion: {standing_avg_range})

## Spatial Patterns

The aim of this section is to visualize the spatial patterns and range of motion during the performance of behavioral gestures. These are the potential insights:

* Identification of specific joint movements that are consistently used during a particular gesture
* Differences in joint movements and range of motion between different performers or performances
* Correlation between certain joint movements and emotional expression conveyed by the gesture
* Identification of unusual or unexpected joint movements that could indicate a unique style or interpretation of the gesture.

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot the movement pathways of each joint
for joint in standing_joint_columns:
    x = df_pose_abs[joint+'_x']
    y = df_pose_abs[joint+'_y']
    z = df_pose_abs[joint+'_z']
    ax.plot(x, y, z, label=joint)

# Set the labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Spatial Pathways of Joints during Behavioral Gestures')

# Set the limits of the plot
max_range = np.array([df_pose_abs[joint+'_x'].max()-df_pose_abs[joint+'_x'].min(),
                      df_pose_abs[joint+'_y'].max()-df_pose_abs[joint+'_y'].min(),
                      df_pose_abs[joint+'_z'].max()-df_pose_abs[joint+'_z'].min()]).max()
for axis in 'xyz':
    eval('ax.set_{}lim(-max_range, max_range)'.format(axis))

# Show the legend and plot
ax.legend()
plt.show()

## Conclusion

(Drafted examples)

* **Timing Variations**: The analysis of duration, tempo, and rhythm revealed interesting differences in the pacing of the gestures across the different categories, which could be indicative of the performer's emotional expression.

* **Flow and Dynamics**: The examination of the smoothness, continuity, and intensity of movements revealed that certain gestures had higher flow values, indicating more dynamic and expressive movements.

* **Body Posture and Alignment**: The assessment of posture and alignment highlighted significant differences in joint movements and range of motion between different gestures, which could be used to identify specific joint movements that are consistently used during a particular gesture.

* **Spatial Patterns**: The visualization of spatial patterns and range of motion during the performance of behavioral gestures revealed unique styles and interpretations of the gestures, which could be correlated with emotional expression conveyed by the gesture.